In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    ### functions에 함수들 넣고, fuction_call을 auto로 해놓으면 알아서 처리해줌.
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")
chain = prompt | llm

response = chain.invoke({"city": "rome"})
response = response.additional_kwargs["function_call"]["arguments"]
response


'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built in 70-80 AD?","answers":[{"answer":"Colosseum","correct":true},{"answer":"Pantheon","correct":false},{"answer":"Roman Forum","correct":false}]},{"question":"Which volcano erupted and buried the city of Pompeii near Rome?","answers":[{"answer":"Mount Vesuvius","correct":true},{"answer":"Mount Etna","correct":false},{"answer":"Mount Olympus","correct":false}]},{"question":"Who was the legendary founder of Rome?","answers":[{"answer":"Romulus","correct":true},{"answer":"Remus","correct":false},{"answer":"Julius Caesar","correct":false}]}]}'

In [3]:
import json

json.loads(response)


{'questions': [{'question': 'What year was Rome founded?',
   'answers': [{'answer': '753 BC', 'correct': True},
    {'answer': '476 AD', 'correct': False},
    {'answer': '1000 AD', 'correct': False}]},
  {'question': 'Who was the first emperor of Rome?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False}]},
  {'question': 'What famous structure in Rome was built in 70-80 AD?',
   'answers': [{'answer': 'Colosseum', 'correct': True},
    {'answer': 'Pantheon', 'correct': False},
    {'answer': 'Roman Forum', 'correct': False}]},
  {'question': 'Which volcano erupted and buried the city of Pompeii near Rome?',
   'answers': [{'answer': 'Mount Vesuvius', 'correct': True},
    {'answer': 'Mount Etna', 'correct': False},
    {'answer': 'Mount Olympus', 'correct': False}]},
  {'question': 'Who was the legendary founder of Rome?',
   'answers': [{'answer': 'Romulus', 'correct': True},
    {'answ

In [ ]:
### 헛소리 줄이고 내 데이터 첨가 가능.
### 프롬프트 상위호환이라고 봄.
### Langchain강의 기준 GPT3, GPT4 모델만 가능함, Ollama같은건 불가.
### 다음은 GPT예제임.
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

# 1. 날씨 API 함수 설명서 (Schema)
weather_function = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "The city name"}
        },
        "required": ["location"],
    },
}

# 2. LLM 객체 생성 (function_call 자동 모드)
llm = ChatOpenAI(temperature=0).bind(
    functions=[weather_function],
    function_call="auto"   # 모델이 필요할 때 함수 호출 JSON을 만들어줌
)

# 3. 프롬프트
prompt = ChatPromptTemplate.from_template("What's the weather in {city}?")
chain = prompt | llm

# 4. 모델 응답 (실제 API 호출이 아님, JSON 구조 반환)
response = chain.invoke({"city": "Rome"})
print("LLM 응답:", response.additional_kwargs)

# 5. JSON 파싱해서 실제 함수 실행 (여기선 가짜 함수)
function_call = response.additional_kwargs["function_call"]
args = json.loads(function_call["arguments"])

def get_current_weather(location):
    # 실제로는 외부 API 호출 → 여기서는 가짜 값 리턴
    return {"location": location, "temperature": "28°C", "condition": "Sunny"}

api_result = get_current_weather(**args)
print("실제 API 결과:", api_result)